# HW 2: Language Modeling

In this homework you will be building several varieties of language models.

## Goal

We ask that you construct the following models in Torch / NamedTensor:

1. A count-based trigram model with linear-interpolation. $$p(y_t | y_{1:t-1}) =  \alpha_1 p(y_t | y_{t-2}, y_{t-1}) + \alpha_2 p(y_t | y_{t-1}) + (1 - \alpha_1 - \alpha_2) p(y_t) $$
2. A neural network language model (consult *A Neural Probabilistic Language Model* http://www.jmlr.org/papers/volume3/bengio03a/bengio03a.pdf)
3. An LSTM language model (consult *Recurrent Neural Network Regularization*, https://arxiv.org/pdf/1409.2329.pdf) 
4. Your own extensions to these models.


Consult the papers provided for hyperparameters.

 


## Setup

This notebook provides a working definition of the setup of the problem itself. You may construct your models inline or use an external setup (preferred) to build your system.

In [4]:
!pip install -q torch torchtext opt_einsum
!pip install -qU git+https://github.com/harvardnlp/namedtensor

In [5]:
import torch
import torchtext
from torchtext.vocab import Vectors

from namedtensor import ntorch
from namedtensor.text import NamedField

import numpy as np

In [6]:
%run models/utils.py

The dataset we will use of this problem is known as the Penn Treebank (http://aclweb.org/anthology/J93-2004). It is the most famous dataset in NLP and includes a large set of different types of annotations. We will be using it here in a simple case as just a language modeling dataset.

To start, `torchtext` requires that we define a mapping from the raw text data to featurized indices. These fields make it easy to map back and forth between readable data and math, which helps for debugging.

In [7]:
# Our input $x$
TEXT = NamedField(names=("seqlen",))

Next we input our data. Here we will use the first 10k sentences of the standard PTB language modeling split, and tell it the fields.

In [8]:
!curl -qO https://raw.githubusercontent.com/harvard-ml-courses/cs287-s18/master/HW2/input.txt
!curl -qO https://raw.githubusercontent.com/harvard-ml-courses/cs287-s18/master/HW2/train.5k.txt
!curl -qO https://raw.githubusercontent.com/harvard-ml-courses/cs287-s18/master/HW2/train.txt
!curl -qO https://raw.githubusercontent.com/harvard-ml-courses/cs287-s18/master/HW2/valid.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  185k  100  185k    0     0   817k      0 --:--:-- --:--:-- --:--:--  926k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  595k  100  595k    0     0  2604k      0 --:--:-- --:--:-- --:--:-- 2756k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 4982k  100 4982k    0     0  5910k      0 --:--:-- --:--:-- --:--:-- 6024k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  390k  100  390k    0     0   498k      0 --:--:-- --:--:-- --:--:--  513k


In [9]:
# Data distributed with the assignment
train, val, test = torchtext.datasets.LanguageModelingDataset.splits(
    path=".", 
    train="train.txt", validation="valid.txt", test="valid.txt", text_field=TEXT)

The data format for language modeling is strange. We pretend the entire corpus is one long sentence.

In [10]:
print('len(train)', len(train))

len(train) 1


Here's the vocab itself. (This dataset has unk symbols already, but torchtext adds its own.)

In [11]:
TEXT.build_vocab(train)
print('len(TEXT.vocab)', len(TEXT.vocab))

len(TEXT.vocab) 10001


When debugging you may want to use a smaller vocab size. This will run much faster.

In [12]:
if False:
    TEXT.build_vocab(train, max_size=1000)
    len(TEXT.vocab)

The batching is done in a strange way for language modeling. Each element of the batch consists of `bptt_len` words in order. This makes it easy to run recurrent models like RNNs. 

In [13]:
from torchtext.data.iterator import BPTTIterator
from torchtext.data import Batch, Dataset
import math
 

class NamedBpttIterator(BPTTIterator):
    def __iter__(self):
        text = self.dataset[0].text
        TEXT = self.dataset.fields['text']
        TEXT.eos_token = None
        text = text + ([TEXT.pad_token] * int(math.ceil(len(text) / self.batch_size)
                                              * self.batch_size - len(text)))
        data = TEXT.numericalize(
            [text], device=self.device)
        data = (data
            .stack(("seqlen", "batch"), "flat")
            .split("flat", ("batch", "seqlen"), batch=self.batch_size)
            .transpose("seqlen", "batch")
        )

        dataset = Dataset(examples=self.dataset.examples, fields=[
            ('text', TEXT), ('target', TEXT)])
        while True:
            for i in range(0, len(self) * self.bptt_len, self.bptt_len):
                self.iterations += 1
                seq_len = min(self.bptt_len, len(data) - i - 1)
                yield Batch.fromvars(
                    dataset, self.batch_size,
                    text = data.narrow("seqlen", i, seq_len),
                    target = data.narrow("seqlen", i+1, seq_len),
                )
                         
            if not self.repeat:
                return

In [14]:
train_iter, val_iter, test_iter = NamedBpttIterator.splits(
    (train, val, test), batch_size=10, bptt_len=32, repeat=False)

Here's what these batches look like. Each is a string of length 32. Sentences are ended with a special `<eos>` token.

In [15]:
it = iter(train_iter)
batch = next(it) 
print("Size of text batch [max bptt length, batch size]", batch.text.shape)
example = batch.text[{"batch": 1}]
print("Second in batch", example)
print("Converted back to string: ", " ".join([TEXT.vocab.itos[i] for i in example.values.data]))

Size of text batch [max bptt length, batch size] OrderedDict([('seqlen', 32), ('batch', 10)])
Second in batch NamedTensor(
	tensor([  72,    5,   28,  247,   61,   12,  216,    5,    0, 1847,   10,    4,
          72,  547,    3, 6506,  163,    7,  105,  479,   38,   31,  295, 4901,
          13,    4,   49,    3,    0,    0,   25, 2471]),
	('seqlen',))
Converted back to string:  shares of its common stock for each of <unk> deposit 's N shares outstanding <eos> liberty national a bank holding company has assets exceeding $ N billion <eos> <unk> <unk> was appointed


The next batch will be the continuation of the previous. This is helpful for running recurrent neural networks where you remember the current state when transitioning.

In [16]:
batch = next(it)
print("Converted back to string: ", " ".join([TEXT.vocab.itos[i] for i in batch.text[{"batch": 1}].values.data]))

Converted back to string:  president and chief executive officer of this financially troubled department store chain effective nov. N succeeding frank robertson who is retiring early <eos> mr. <unk> was previously president and chief operating officer


The batch object also contains the targets for the given batch in the field `target`. The target is simply the text offset by one.

In [17]:
print("Converted back to string: ", " ".join([TEXT.vocab.itos[i] for i in batch.target.get("batch", 1).values.data]))

Converted back to string:  and chief executive officer of this financially troubled department store chain effective nov. N succeeding frank robertson who is retiring early <eos> mr. <unk> was previously president and chief operating officer of


## Assignment

Now it is your turn to build the models described at the top of the assignment. 

Using the data given by this iterator, you should construct 3 different torch models that take in batch.text and produce a distribution over the next word. 

When a model is trained, use the following test function to produce predictions, and then upload to the kaggle competition: https://www.kaggle.com/c/harvard-cs287-s19-hw2/

For the final Kaggle test, we will have you do a next word prediction task. We will provide a 10 word prefix of sentences, and it is your job to predict 20 possible next word candidates

In [18]:
!head input.txt

but while the new york stock exchange did n't fall ___
some circuit breakers installed after the october N crash failed ___
the N stock specialist firms on the big board floor ___
big investment banks refused to step up to the plate ___
heavy selling of standard & poor 's 500-stock index futures ___
seven big board stocks ual amr bankamerica walt disney capital ___
once again the specialists were not able to handle the ___
<unk> james <unk> chairman of specialists henderson brothers inc. it ___
when the dollar is in a <unk> even central banks ___
speculators are calling for a degree of liquidity that is ___


As a sample Kaggle submission, let us build a simple unigram model.  

In [19]:
from collections import Counter
count = Counter()
for b in iter(train_iter):
    count.update(b.text.values.contiguous().view(-1).tolist())
count[TEXT.vocab.stoi["<eos>"]] = 0
predictions = [TEXT.vocab.itos[i] for i, c in count.most_common(20)]
with open("sample.txt", "w") as fout:
    print("id,word", file=fout)
    for i, l in enumerate(open("input.txt"), 1):
        print("%d,%s"%(i, " ".join(predictions)), file=fout)


The metric we are using is mean average precision of your 20-best list. 

$$MAP@20 = \frac{1}{|D|} \sum_{u=1}^{|D|} \sum_{k=1}^{20} Precision(u, 1:k)$$

Ideally we would use log-likelihood or ppl as discussed in class, but this is the best Kaggle gives us. This takes into account whether you got the right answer and how highly you ranked it. 

In particular, we ask that you do not game this metric. Please submit *exactly 20* unique predictions for each example.


As always you should put up a 5-6 page write-up following the template provided in the repository: https://github.com/harvard-ml-courses/nlp-template

In [20]:
%run models/lstm.py

In [21]:
lstm = LSTM(TEXT, embedding_dim=100, hidden_dim=100)
optim = torch.optim.Adam(lstm.parameters(), lr=1)

In [22]:
lstm(batch.text).shape

OrderedDict([('batch', 10), ('seqlen', 32), ('vocab', 10001)])

In [23]:
batch.target.shape

OrderedDict([('seqlen', 32), ('batch', 10)])

In [24]:
 _, best_words = ntorch.topk(lstm(batch.text)[{'seqlen': -1}],
                                            'vocab', 20)

In [25]:
last_target = batch.target[{'seqlen': -1}]

In [26]:
last_target

NamedTensor(
	tensor([2171,    5,    8,    2,  128,   18,  300,    6,    6,  303]),
	('batch',))

In [ ]:
def kaggle_loss(model, batch):
     _, best_words = ntorch.topk(model(batch.text)[{'seqlen': -1}], 'vocab', 20)
    last_target = batch.target[{'seqlen': -1}]
    is_correct = best_words == last_target
    scores = torch.cumsum(is_correct.values, 1).float() / (1 + torch.arange(20)).float()
    return scores.sum()

In [35]:
best_words[{'vocab': 0}] = last_target

In [36]:
is_correct = best_words == last_target

In [38]:
scores = torch.cumsum(is_correct.values, 1).float() / (1 + torch.arange(20)).float()

In [42]:
scores.sum()

tensor(35.9774)

In [31]:
torch.arange(20)+1

tensor([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
        19, 20])

In [33]:
best_words.shape

OrderedDict([('batch', 10), ('vocab', 20)])

In [45]:
def cb(**kwargs):
    ep, t, v = (kwargs['epoch'], (kwargs['train_loss'] / len(train_iter)), (kwargs['val_loss'] / len(val_iter)))
    print(f'Epoch {ep}, train loss = {t:.2f} [{np.exp(t):.2f}], val loss = {v:.2f} [{np.exp(v):.2f}]')

train_model(lstm, lstm_loss, optim, 
        train_iter, val_iter, num_epochs=15, 
        callback=cb,
        progress_bar=True)

KeyboardInterrupt: 

In [37]:
import azureml

ModuleNotFoundError: No module named 'azureml'

In [36]:
!pip install azureml-sdk

  Using cached https://files.pythonhosted.org/packages/ff/08/042fc0fc6126054869fc087998184ea7c99ff6db8a0aee0bcb7d9a82a090/azureml_sdk-1.0.15.1-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/cc/eb/dce049020d70c4322bb6fc4b1cfd2ec7915ba88802509a26d0751b3b76c5/azureml_train-1.0.15-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/a6/8e/3765eab01a13db1b32b1f6e4a8ec6b897ea329eee5c7028cfe983008ab6e/azureml_pipeline-1.0.15-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/a9/a6/dd85766160ec29c31200c15b88756fe4fceb8681396e72b2afa32cda712e/azureml_core-1.0.15-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/4a/77/21a8b6a1b7bb682deb6060f68e04863b6cb0a0883097fed4f37cdc04c48d/azureml_train_core-1.0.15-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/4d/b0/6e9b05d46859d942e41bd1718a29c55e00f89aa41075c173c81c1ec76d78/azureml_pipeline_steps-1.0.15-py3-none-any.whl
  Using cached https

In [30]:
from azureml.core import Experiment

ModuleNotFoundError: No module named 'azureml'